### Actividad Integradora - Equipo 2 - Sistemas Multiagentes
18 de noviembre de 2021, Tecnológico de Monterrey

<br>

Se simula a través de un sistema multiagentes un almacen necesitando recoger cajas en pilas con la ayuda de robots inteligentes. Utilizando un túnel HTTP, envía los datos obtenidos a Unity para que sean visualizados en un espacio tridimensional.

In [ ]:
#@title Imports e Instalaciones

# Instalación de paquetes externos a las librerías estándar
%pip install mesa pyngrok --quiet

# Paquete esencial que ayuda a modelar sistemas multiagentes
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

# Paquete matemático utilizado para matrices de declaración sencilla
import numpy as np

# Paquetes útiles para trabajar y graficar la animación de la simulación
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

# Nativo de Python para aleatorizar la aparición de las cajas
import random

# Nativos de Python para medir la duración de las simulaciones
import time
import datetime

# Paquetes para la transferencia de datos por HTTP
from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import json
import os

# Configuración adicional que se instala de ngrok
ngrok.install_ngrok()

     |████████████████████████████████| 668 kB 9.3 MB/s 
     |████████████████████████████████| 745 kB 43.4 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 


In [ ]:
#@title Funciones Auxiliares

# Conversión de formato lista de acciones a JSON
def actions_to_dictlist(actions):
  actions_list = []
  for action in actions:
    actions_list.append({
        "type": action[0],
        "robotID": action[1],
        "dx": action[2][0],
        "dy": action[2][1],
        "stackSize": action[3]
    })
  return actions_list

In [ ]:
#@title Función Recolectora

# Función auxiliar para capturar el modelo en un momento como datos
# Se almacenan: {0: Piso vacío, 1: Pila vacía, 2-6: Numero de cajas, 7: Robot, 8: Pared}
def get_grid(model):
  # Itera el modelo llenando una cuadrícula que inicia vacía
  grid = np.zeros((model.grid.width, model.grid.height))
  for (cell_content, x, y) in model.grid.coord_iter():
    if len(cell_content) > 1:
      # Solo hay múltiples agentes en una celda cuando se trata de un robot sobre piso
      grid[x][y] = 7
    elif cell_content[0].cell_type == 0:
      # Celda de tipo pared
      grid[x][y] = 8
    elif cell_content[0].boxes_here > 0:
      # Si hay cajas, así se pintan
      grid[x][y] = cell_content[0].boxes_here + 1
    else:
      # Pila vacía o piso vacío
      grid[x][y] = cell_content[0].cell_type - 1
  # Transposición para que (x,y) queden como cartesianas y se anime Width*Height
  return np.transpose(grid)

In [ ]:
#@title Clase Celda

# Clase celda para identificar la interacción de los robots con esta posición
class Cell(Agent):
  # Constructor
  def __init__(self, id, model, cell_type, num_boxes):
    # Construcción de la clase padre Agent
    super().__init__(id, model)
    self.id = id

    # Tipo de la celda {0: Pared, 1: Piso, 2: Pila}
    self.cell_type = cell_type
    self.boxes_here = num_boxes

In [ ]:
#@title Clase Robot

# Clase de un agente robot inteligente para la limpieza del almacen
class Robot(Agent):
  # Variable estática o de la clase auxiliar para rodear obstaculos
  moves = [(-1,-1), (0,-1), (1,-1), (1,0), (1,1), (0,1), (-1,1), (-1,0)]
  move_counter = 0

  # Constructor
  def __init__(self, id, model, spawn, end):
    # Construcción de la clase padre Agent
    super().__init__(id, model)
    self.id = id

    # Variables de ubicación y desplazamiento
    self.pos = spawn
    self.destination = end
    self.end_pos = end
    self.last_action = ["Reposar", (0,0), -1]

    # Variables asociadas con la tarea de llevar una caja
    self.assigned_box = None
    self.assigned_stack = None
    self.carrying_box = False
    self.active = True

  # Momento de acción - Movimiento del robot para completar su tarea
  def step(self):
    # Evita el step si el robot ya terminó sus tareas
    if not(self.active):
      self.last_action = ["Reposar", (0,0), -1]
      return

    # Elige la dirección de movimiento, dando prioridad a las diagonales
    dx = self.destination[0] - self.pos[0]
    dy = self.destination[1] - self.pos[1]
    move = (0 if dx == 0 else dx // abs(dx), 0 if dy == 0 else dy // abs(dy))
    next_pos = (self.pos[0] + move[0], self.pos[1] + move[1])

    # Primer caso, se llegaría por una caja. No se mueve, solo la recoge
    if next_pos == self.assigned_box and not(self.carrying_box):
      # Actualiza el estado de la tarea y "quita" la caja de esa celda
      self.carrying_box = True
      self.assigned_box = None
      self.destination = self.assigned_stack
      self.model.grid.get_cell_list_contents(next_pos)[0].boxes_here = 0
      self.last_action = ["Recoger", move, -1]
    # Segundo caso, se llegaría a una pila de cajas. No se mueve, solo la deja
    elif next_pos == self.assigned_stack and self.carrying_box:
      # Actualiza el estado de la tarea y "deja" la caja en la pila
      self.carrying_box = False
      self.assigned_stack = None
      self.destination = self.end_pos
      self.model.grid.get_cell_list_contents(next_pos)[0].boxes_here += 1
      self.model.robots_free.add(self)
      self.last_action = ["Dejar", move,
        self.model.grid.get_cell_list_contents(next_pos)[0].boxes_here]
    else:
      # Tercer caso, podría chocar con cajas no asignadas, muros u otro robot
      agents_there = self.model.grid.get_cell_list_contents(next_pos)
      
      # Considera la necesidad de rodear según lo ubicado en esa posición
      if (len(agents_there) > 1 or agents_there[0].cell_type in [0,2] or
          agents_there[0].boxes_here > 0):
        next_pos = self.go_around(next_pos, move)
      
      # No hace movimiento si realmente está atorado y no supo rodear
      final_move = (next_pos[0] - self.pos[0], next_pos[1] - self.pos[1])
      self.last_action = ["Moverse", final_move, -1]
      
      # Tercer y cuarto paso (movimiento regular), se avanza a la posición inteligente
      self.model.grid.move_agent(self, next_pos)
      self.pos = next_pos
      Robot.move_counter += 1

    if self.pos == self.end_pos and self.destination == self.end_pos:
      self.model.active_robots -= 1
      self.active = False

  # Nota: El movimiento se hace en step() y no advance() para evitar que dos
  # robots independientes decidan por la misma celda y choquen. Esto inherentemente
  # deja con prioridad de movimiento a los primeros robots agregados al schedule
  
  # Al fin de una subasta del modelo, un robot registra la tarea que le ocupa
  def assign_box(self, box_pos, stack_pos):
    self.destination = box_pos
    self.assigned_box = box_pos
    self.assigned_stack = stack_pos

  # Elige una posición alternativa dado que la elegida está obstaculizada
  def go_around(self, next_pos, move):
    # Inicializa el valor de costo como uno máximo para que sea reemplazado
    move_cost = self.model.n + self.model.m + 1
    move_idx = self.moves.index(move)

    # Se intenta un máximo de 8 rotaciones (4 derecha, 4 izquierda)
    for rotations in range(1, 5):
      # Elige los movimientos rotados
      right_move = self.moves[(move_idx + rotations) % len(self.moves)]
      left_move = self.moves[(move_idx - rotations + len(self.moves)) % len(self.moves)]

      # Encuentra la posición donde quedaría
      right_pos = (self.pos[0] + right_move[0], self.pos[1] + right_move[1])
      left_pos = (self.pos[0] + left_move[0], self.pos[1] + left_move[1])

      # Calcula el costo de moverse al lugar alrededor
      right_cost = self.go_around_cost(right_pos, move_cost)
      left_cost = self.go_around_cost(left_pos, move_cost)

      # Se elige el mejor escenario de los anteriores, si es que alguno sirve
      if right_cost != move_cost or left_cost != move_cost:
        min_cost = min(right_cost, left_cost)
        return right_pos if min_cost == right_cost else left_pos
    
    # Dado que no fue posible elegir una alternativa, el robot se esperará.
    # Considerando que los obstaculos, cajas como robots se terminarán moviendo
    # el robot solo se quedará estancando de manera temporal
    return self.pos

  # Auxiliar al método de rodear, checa si una opción es viable y con que costo
  def go_around_cost(self, new_pos, default_cost):
    # Define la viabilidad con lo obstaculizado que este la posición
    agents_there = self.model.grid.get_cell_list_contents(new_pos)
    if (len(agents_there) > 1 or agents_there[0].cell_type in [0,2] or
        agents_there[0].boxes_here > 0):
      return default_cost
    # Como no hay obstaculos, el criterio de decisión o costo, será la distancia
    return self.model.distance(new_pos, self.destination)

In [ ]:
#@title Clase Modelo

class StorageModel(Model):
  # Constructor
  def __init__(self, m, n, num_robots, num_boxes, max_duration):
    # Inicialización de atributos para almacenar los datos recibidos
    self.m = m
    self.n = n
    self.num_robots = num_robots
    self.active_robots = num_robots
    self.num_boxes = num_boxes
    self.robots = []
    
    # Creacíon de un Multigrid() para poder tener más de un agente por celda
    # Se incluye un padding de 1 espacio por lado para mostrar las paredes
    self.grid = MultiGrid(m + 2, n + 2, False)

    # Permite activar al mismo tiempo todos los componentes del modelo
    self.schedule = SimultaneousActivation(self)
    self.max_duration = max_duration

    # Recolector de datos para futura representación gráfica
    self.grid_collector = DataCollector(model_reporters = {"Grid": get_grid})

    # Se generan las posiciones de las pilas
    num_stacks = int(np.ceil(self.num_boxes / 5))
    self.stacks_pos = [(i % n + 1, i // n + 1) for i in range(num_stacks)]

    # Variables para las subastas de las tareas de llevar cajas
    self.boxes_left = set()
    self.robots_free = set()
    self.stack_to_assign = [0, 0]

    # Se genera el terreno, las celdas con sus diferentes tipos
    for (content, x, y) in self.grid.coord_iter():
      # Paredes en los límites del tablero
      if x in [0, m + 1] or y in [0, n + 1]:
        new_cell = Cell((x,y), self, 0, 0)
      # Pilas donde se colocarán las cajas
      elif (x,y) in self.stacks_pos:
        new_cell = Cell((x,y), self, 2, 0)
      # Piso donde pueden andar los robots o inicializarse cajas
      else:
        new_cell = Cell((x,y), self, 1, 0)
      self.grid.place_agent(new_cell, (x,y))
    
    # Se seleccionan aleatoriamente celdas restantes para cajas y robots
    free_cells = [(x, y) for (content, x, y) in self.grid.coord_iter() if (x,y)
      not in self.stacks_pos and x not in [0, m + 1] and y not in [0, n + 1]]
    selected_spawns = random.sample(free_cells, self.num_boxes + self.num_robots)

    # Se guardan puntos para dejar los robots al terminar las tareas
    parking_sep = m // self.num_robots
    robot_ends = [(m - ((i * parking_sep) % m), n - (
        (i * parking_sep) // m)) for i in range(self.num_robots)]

    # Usa las primeras celdas libres para los robots
    for i in range(self.num_robots):
      new_robot = Robot(i, self, selected_spawns[i], robot_ends[i])
      self.grid.place_agent(new_robot, (selected_spawns[i]))
      self.schedule.add(new_robot)
      self.robots_free.add(new_robot)
      self.robots.append(new_robot)
    
    # Usa el resto de celdas aleatorias para las cajas
    for i in range(self.num_robots, len(selected_spawns)):
      self.grid.get_cell_list_contents(selected_spawns[i])[0].boxes_here = 1
      self.boxes_left.add(selected_spawns[i])

    # Recolección inicial, antes de cualquier recogida instantánea de cajas
    self.grid_collector.collect(self)

  # Unidad de cambio del modelo. También se llama a actuar a los agentes
  def step(self):
    self.auction_boxes()
    self.schedule.step()
    self.grid_collector.collect(self)

  def auction_boxes(self):
    # Subasta todas las cajas que quedan sin ser asignadas
    for box_pos in self.boxes_left.copy():

      # Verifica que sigan habiendo robots disponibles para la subasta
      if len(self.robots_free) == 0: return
      
      # Abre la subasta a todos los robots desocupados
      chosen_robot = None
      task_cost = self.m * self.n
      for robot in self.robots_free:
        robot_cost = (self.distance(robot.pos, box_pos) + 
          self.distance(box_pos, self.stacks_pos[self.stack_to_assign[0]]))
        # Almacena al robot que menos le cueste la tarea
        if task_cost > robot_cost:
          task_cost = robot_cost
          chosen_robot = robot
      
      # Asigna la tarea y deja de considerar a esa caja, ese robot y quizá a la pila
      chosen_robot.assign_box(box_pos, self.stacks_pos[self.stack_to_assign[0]])
      self.boxes_left.remove(box_pos)
      self.robots_free.remove(chosen_robot)
      self.stack_to_assign[1] += 1
      if self.stack_to_assign[1] == 5:
        self.stack_to_assign = [self.stack_to_assign[0] + 1, 0]

  # Distancia entre dos puntos en una cuadrilla con movimientos diagonales
  # Se utiliza la distancia de Chebyshev o del tablero de ajedrez
  def distance(self, pos1, pos2):
    dx = abs(pos1[0] - pos2[0])
    dy = abs(pos1[1] - pos2[1])
    return max(dx, dy)

  # Implementación más sencilla de un recolector de datos
  # Conseguirá para Unity los últimos movimientos de los robots
  def report_moves(self):
    actions = list()
    for robot in self.robots:
    # Se agrega a la lista Ej. ["Recoger", 3, (1,1), -1]
      actions.append([robot.last_action[0], robot.id,
                      robot.last_action[1], robot.last_action[2]])
    return actions

In [ ]:
#@title Clase Servidor de la simulación

# Clase que maneja las requests al servidor: envía y recibe datos
# Controla la simulación según lo vaya pidiendo Unity
class SimulationServer(BaseHTTPRequestHandler):
  # Modelo de MESA que se simulará, activación del logging
  model = None
  log_active = False
  start_time = None

  # Manejo de un método GET enviado al servidor
  def do_GET(self):
    self.log("GET")
    self._set_response()
    self.wfile.write(f"GET request for {self.path}".encode('utf-8'))


  # Manejo de un método POST enviado al servidor
  def do_POST(self):
    # Lectura del método POST que ha llegado con un request
    content_length = int(self.headers['Content-Length'])
    post_data = json.loads(self.rfile.read(content_length))
    self.log("POST", post_data)
    
    # Selección de la respuesta según la petición. Envío codificado
    response = self.choose_response(post_data["request"])
    self._set_response()
    self.wfile.write(response.encode('utf-8'))
    if response == "{\"order\": \"stop\"}":
      raise KeyboardInterrupt()


  def choose_response(self, request):
    # Variables de trabajo. Se devuelve la transposición de get_grid
    response = {"data": ""}

    # Árbol de respuestas de Python para Unity según el request
    # Da prioridad a enviar un stop si se alcanza el tiempo máximo
    if time.time() - self.start_time > self.model.max_duration:
      response = {"order" : "stop"}
    elif request == "board-init":
      response = {"m": self.model.m, "n": self.model.n}
    elif request == "robots":
      response = {"Items" : [{"id":robot.id, "x":robot.pos[1],
         "z":robot.pos[0]} for robot in self.model.robots]}
    elif request == "stacks":
      response = {"Items": [{"x":y, "z":x, "boxes":0} for (x,y)
        in self.model.stacks_pos]}
    elif request == "boxes":
      response = {"Items": [{"x":y, "z":x} for (x,y) in
          self.model.boxes_left]}
    elif request == "step":
      self.model.step()
      step_actions = self.model.report_moves()
      response = {"Items": actions_to_dictlist(step_actions)}

    return json.dumps(response)
        

  # Configura una respuesta HTTP de éxito con encabezado
  def _set_response(self):
    self.send_response(200)
    self.send_header('Content-type', 'text/html')
    self.end_headers()

  # Logging de la clase padre sobrescrito para quitar logs
  def log_message(self, format, *args):
    if (self.log_active):
      super().log_message(format, *args)

  # Log de métodos HTTP por logging
  def log(self, method, data = None):
    # Requiere activación
    if not(self.log_active): return
    if method == "GET":
      logging.info("GET request,\nPath: %s\nHeaders:\n%s\n",
      str(self.path), str(self.headers))
    elif method == "POST":
      logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
      str(self.path),str(self.headers), json.dumps(data))

In [ ]:
#@title Asignación del modelo

# Dado que el servidor recibe una clase y no un objeto, se tiene
# que atar al modelo multi-agentes como una variable estática
def attach_model(simulation_server, model_params):
  new_model = StorageModel(*model_params)
  simulation_server.model = new_model

In [ ]:
#@title Run del servidor

# Run para el servidor, habiendo creado el servidor que conoce al modelo
def run(server_class, handler_class, port, log):
    # Asegura solo tener un servidor a la vez, matando a los demás
    ngrok.kill()
    
    # Crea el servidor y lo conecta al túnel en el puerto abierto
    httpd = server_class(('', port), handler_class)
    public_url = ngrok.connect(port).public_url
    
    # Impresión para monitorización y guía al usuario final
    if log:
      logging.basicConfig(level = logging.INFO)
      logging.info("Iniciando httpd...\n")
    print("\nServidor corriendo. Ctrl+C para detener conexión y continuar")
    print(f"Túnel público: {public_url}")
    print(f"Túnel local: http://127.0.0.1:{port}")

    # Corre el servidor hasta cualquier excepción, de teclado o de fin
    handler_class.start_time = time.time()
    try: httpd.serve_forever()
    except: pass
    handler_class.end_time = time.time()

    # Cierre del servidor
    httpd.server_close()
    if log: logging.info("Deteniendo httpd...\n")
    print("Servidor detenido. Continúa la animación en Colab")

In [ ]:
#@title Estadísticas de ejecución

# Impresión de los datos relevantes para MAS
def show_statistics(simulation_server):
  # Extracción de los tiempos de ejecución
  print("\nEstadísticas de la ejecución")
  max_duration = simulation_server.model.max_duration
  duration = simulation_server.end_time - simulation_server.start_time

  # Formateo y restricción a un valor máximo del tiempo de ejecución
  if (duration >= max_duration):
    print(f"Tiempo de ejecución: {max_duration}s (Duración máxima permitida)")
  else:
    print(f"Tiempo de ejecución: {round(duration, 3)}s (Tarea terminada antes de tiempo)")

  # Impresión del contador que une los movimientos de todos los robots
  print("Movimientos totales de Robot: ", Robot.move_counter, '\n')

In [ ]:
#@title Animación

# Genera una animación de un modelo que recolecta sus cuadrículas
def animate_simulation(model):
  # Recopila los datos del recolector por ser animados
  grids = model.grid_collector.get_model_vars_dataframe()
  
  # Creación de un mapa de colores
  # {0: Piso vacío, 1: Pila vacía, 2-6: Numero de cajas, 7: Robot, 8: Pared}
  colors = [(256,256,256), (200,200,200), (178,155,129), (152, 128, 100),
    (126, 101, 73), (99, 73, 44), (73, 46, 16), (27, 52, 25), (65, 65, 65)]
  
  # Genera los objetos de color con numpy para matplotlib
  storage_colors = [[r / 256, g / 256, b / 256, 1] for (r,g,b) in colors]
  storage_cmap = matplotlib.colors.ListedColormap(storage_colors)

  # Modificación de parámetros de matplotlib para una impresión mejor
  plt.rcParams["animation.html"] = "jshtml"
  plt.rcParams["axes.titlesize"] = 28
  matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

  # Construcción y personalización de la gráfica animada
  fig, axs = plt.subplots(figsize=(7, 7))
  axs.set_title("Storage Room Simulation")
  patch = plt.imshow(grids.iloc[0][0], cmap = storage_cmap)

  # Creación y ejecución del objeto animación
  storage_simulation = animation.FuncAnimation(fig,
    lambda i: patch.set_data(grids.iloc[i][0]) , frames = grids.size)
  return storage_simulation

In [ ]:
#@title Flujo principal del programa

# Parámetros de la simulación
M, N = [12, 8]
NUM_ROBOTS = 5
NUM_BOXES = 6
MAX_DURATION = 3600

# Ejecución de un modelo desde un servidor, animación final
model_params = [M, N, NUM_ROBOTS, NUM_BOXES, MAX_DURATION]
attach_model(SimulationServer, model_params)
run(HTTPServer, SimulationServer, port = 8585, log = False)
show_statistics(SimulationServer)
animate_simulation(SimulationServer.model)


Servidor corriendo. Ctrl+C para detener conexión y continuar
Túnel público: http://d478-35-245-231-88.ngrok.io
Túnel local: http://127.0.0.1:8585
